In [132]:
import json
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [133]:

def obter_dados(fonte, fonte_amigos=None):
    if fonte == 'json':
        with open('cursos.json') as f:
            dados = json.load(f)
    elif fonte == 'webhook':
        url = 'https://exemplo.com/api/dados'
        response = requests.get(url)
        dados = response.json()
    else:
        raise ValueError('Fonte de dados inválida')

    if fonte_amigos == 'json':
        with open('amigos.json') as f:
            amigos = json.load(f)
    elif fonte_amigos == 'webhook':
        url = 'https://exemplo.com/api/amigos'
        response = requests.get(url)
        amigos = response.json()
    else:
        amigos = []

    return dados, amigos

In [134]:
def obter_cursos_avaliados_usuario(usuario_id, df):
    return df[df['id'] == usuario_id]['curso_nome'].unique()

In [135]:
def obter_cursos_avaliados_amigos(usuario_id, df):
    amigos_avaliaram = df[df['amigo_id'].notnull()]['amigo_id'].unique()
    amigos_cursos = []
    for amigo in amigos_avaliaram:
        cursos_avaliados_amigo = df[(df['amigo_id'] == amigo) & (df['id'] != usuario_id)][['curso_nome', 'amigo_curso_avaliacao']]
        if not cursos_avaliados_amigo.empty:
            cursos_avaliados_amigo = cursos_avaliados_amigo.groupby('curso_nome').mean()
            cursos_avaliados_amigo['usuario_avaliacao'] = df[(df['id'] == usuario_id) & (df['curso_nome'].isin(cursos_avaliados_amigo.index))]['curso_avaliacao'].values
            cursos_avaliados_amigo['amigo_media_avaliacao'] = df[df['id'] == amigo]['curso_media_avaliacao'].values[0]
            amigos_cursos.append(cursos_avaliados_amigo)
    if not amigos_cursos:
        return pd.DataFrame(columns=['curso_nome', 'usuario_avaliacao', 'amigo_media_avaliacao', 'amigo_curso_avaliacao'])
    return pd.concat(amigos_cursos).reset_index()


In [136]:
def recomendar_cursos(avaliacoes_usuario, avaliacoes_amigos):
    df = pd.concat([avaliacoes_usuario, avaliacoes_amigos], ignore_index=True)
    df.drop_duplicates(subset=['curso_nome'], keep='first', inplace=True)

    X = df[['amigo_media_avaliacao', 'usuario_avaliacao']]
    y = df['curso_media_avaliacao']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    reg = LinearRegression().fit(X_train, y_train)

    cursos_recomendados = list(reg.predict(X_test))
    cursos_recomendados_df = pd.DataFrame({'curso_nome': X_test.index, 'curso_media_avaliacao_recomendada': cursos_recomendados})
    cursos_recomendados_df.sort_values('curso_media_avaliacao_recomendada', ascending=False, inplace=True)

    return cursos_recomendados_df


In [138]:
fonte = input('Digite a fonte dos dados (json ou webhook): ')
fonte_amigos = input('Digite a fonte dos amigos (json, webhook ou nenhum): ')
dados, amigos = obter_dados(fonte, fonte_amigos)
df = pd.DataFrame(columns=['id', 'nome', 'idade', 'curso_nome', 'curso_categoria', 'curso_avaliacao', 'amigo_id', 'amigo_nome', 'amigo_curso_nome', 'amigo_curso_avaliacao'])
rows = []
for usuario in dados['usuarios']:
    for curso in usuario['cursos']:
        row = {
            'id': usuario['id'],
            'nome': usuario['nome'],
            'idade': usuario['idade'],
            'curso_nome': curso['nome'],
            'curso_categoria': curso['categoria'],
            'curso_avaliacao': curso['avaliacao'],
            'amigo_id': None,
            'amigo_nome': None,
            'amigo_curso_nome': None,
            'amigo_curso_avaliacao': None
        }
        rows.append(row)
        if 'amigos' in usuario:
            for amigo in usuario['amigos']:
                for amigo_curso in amigo['cursos']:
                    row = {
                        'id': usuario['id'],
                        'nome': usuario['nome'],
                        'idade': usuario['idade'],
                        'curso_nome': curso['nome'],
                        'curso_categoria': curso['categoria'],
                        'curso_avaliacao': curso['avaliacao'],
                        'amigo_id': amigo['id'],
                        'amigo_nome': amigo['nome'],
                        'amigo_curso_nome': amigo_curso['nome'],
                        'amigo_curso_avaliacao': amigo_curso['avaliacao']
                    }
                    rows.append(row)
df = pd.DataFrame(rows)


In [139]:
df.drop_duplicates(inplace=True)

In [140]:
cursos_agrupados = df.groupby(['curso_nome', 'curso_categoria'])['curso_avaliacao'].mean().reset_index()

In [141]:
cursos_agrupados.rename(columns={'curso_avaliacao': 'curso_media_avaliacao'}, inplace=True)

In [142]:
df = df.merge(cursos_agrupados, on=['curso_nome', 'curso_categoria'], how='left')

In [143]:
df.sort_values(['nome', 'curso_media_avaliacao'], ascending=[True, False], inplace=True)

In [ ]:
print(df.head())

In [144]:
def recomendar_cursos(avaliacoes_usuario, avaliacoes_amigos):
    avaliacoes = pd.concat([avaliacoes_usuario] + avaliacoes_amigos, axis=0)

    encoder = LabelEncoder()
    avaliacoes['curso_categoria_enc'] = encoder.fit_transform(avaliacoes['curso_categoria'])

    cursos_avaliados = avaliacoes_usuario['curso_nome']
    cursos_nao_avaliados = avaliacoes['curso_nome'][~avaliacoes['curso_nome'].isin(cursos_avaliados)]
    
    cursos_nao_avaliados = avaliacoes[~avaliacoes['curso_nome'].isin(cursos_avaliados)]['curso_nome'].unique()
    
    cursos_nao_avaliados = pd.DataFrame({'curso_nome': cursos_nao_avaliados})
    cursos_nao_avaliados = cursos_nao_avaliados.merge(
        avaliacoes.groupby('curso_nome')['curso_avaliacao'].mean(),
        on='curso_nome',
        how='left'
    )
    cursos_nao_avaliados = cursos_nao_avaliados.merge(
        avaliacoes.groupby('curso_nome')['curso_categoria_enc'].first(),
        on='curso_nome',
        how='left'
    )

    X_train, y_train = avaliacoes[['curso_media_avaliacao', 'curso_categoria_enc']], avaliacoes['curso_avaliacao']
    X_test = cursos_nao_avaliados[['curso_media_avaliacao', 'curso_categoria_enc']]
    
    reg = LinearRegression()
    reg.fit(X_train, y_train)
    
    cursos_nao_avaliados['predicted_rating'] = reg.predict(X_test)
    
    cursos_nao_avaliados.sort_values(by='predicted_rating', ascending=False, inplace=True)
    
    return cursos_nao_avaliados['curso_nome'][:5].tolist()


In [145]:
avaliacoes_usuario = obter_cursos_avaliados_usuario('usuario1', df)
amigos_avaliaram = df[df['amigo_id'].notnull()]['amigo_id'].unique()
avaliacoes_amigos = obter_cursos_avaliados_amigos('usuario1', df) if len(amigos_avaliaram) > 0 else pd.DataFrame()
cursos_recomendados = recomendar_cursos(avaliacoes_usuario, avaliacoes_amigos)
print(cursos_recomendados)


KeyError: 'usuario_id'